In [ ]:
%cd ..
%env TOKENIZERS_PARALLELISM=false

In [ ]:
import torch
from tqdm.auto import tqdm

from src import DataLoader, PredictionDataset, TokenClassificationModel
from src.scoring import compute_score

#### 設定各項參數

In [ ]:
device = torch.device('cuda')
model_path = 'weights/tc.pt'
dataset_path = 'data/splitted/test.csv'
batch_size = 1

#### 載入模型

In [ ]:
model = TokenClassificationModel()
model.load_state_dict(torch.load(model_path))
model = model.requires_grad_(False).to(device)

#### 載入資料集

In [ ]:
dataset = PredictionDataset(dataset_path, model.tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, pin_memory=True)

#### 進行預測

In [ ]:
answers = []
for xs, encodings, state in tqdm(dataloader):
    encodings.__setstate__(state)
    encodings = encodings.to(device)

    special_tokens_mask = encodings.pop('special_tokens_mask')

    preds = model(encodings)
    ans = model.decode_answers(xs, encodings, special_tokens_mask, preds)
    answers.extend(ans)

#### 計算分數

In [ ]:
compute_score(answers)